In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(r"C:\Users\Owner\Dropbox\Proj\LIC_Development")
taxrolls = pd.read_parquet('taxrolls0924.parquet.gz')

In [2]:
taxrolls.head()

,BORO,BLOCK,LOT,EASE,TC,FMV,UNITS,BC,SQFT,ZIP,LAND_AREA,is_coop_or_condo,is_condo,is_coop,fy,TCfull
0,1.0,1.0,10.0,,4,337093000.0,1.0,Y4,2542066.0,10004,7665016.0,False,False,False,2024,4
1,1.0,1.0,101.0,,4,30489000.0,0.0,P7,541886.0,10004,541886.0,False,False,False,2024,4
2,1.0,1.0,111.0,,4,8675215.0,1.0,Y4,47580.0,10004,64451.0,False,False,False,2024,4
3,1.0,1.0,112.0,,4,378547.0,1.0,Y4,8445.0,10004,7225.0,False,False,False,2024,4
4,1.0,1.0,201.0,,4,233580000.0,0.0,Z9,603130.0,10004,2764190.0,False,False,False,2024,4


# Introduction

Long Island City (LIC), a neighborhood in Queens, experienced substantial real estate development in the early 21st century. The development can be observed in time lapse Google Earth images of the LIC downtown, centered around the Queens Plaza subway station (see images below). In this report, I investigate the character of this development. I consider the number of properties, the number of units, full market value, and total building square footage, by type of property. 

To begin, I explore the extent of LIC development, compared to other zip codes in the city. I find that "core LIC", which I defined to be zip code 11101, had the third largest increase in gross square footage among zip codes in the City between the years of 2009 through 2024 (years refer to City fiscal years, and therefore estimates on the tax rolls are as of the prior January, for example when I say 2009, the estimates are intended to be as of January, 2008). The only two zip codes with larger increases were 10001, which includes the Hudson Yards development, and 11201, which includes Downtown Brooklyn and DUMBO. 

Digging into the particulars of LIC development itself, I am interested in what kinds of residential developments occurred in LIC over this period. In particular, what share of new properties are owner-occupied condos versus renter-occupied units? It is challenging to estimate these shares using the available data, since rental units can be "condo'ed out," so the ownership structure does not necessarily imply owner-occupancy. It is also challenging to separate pied-à-terre condos from rental units (as they will both look like non-owner occupancy). Nevertheless, using the available data I attempt to identify condo ownership, and owner-occupied condos, separately from rental units, and report on the character of residential development. 

# Overall Development of Real Estate Square Footage by Zip Code

In [3]:
dat = taxrolls.copy()
dat['TC'] = dat['TC'].astype(str)
dat['ZIP'] = dat['ZIP'].astype(str).str[0:5]
dat['lic_proper'] = dat['ZIP']=='11101'
dat['lic_broad'] = dat['ZIP'].isin(['11101','11102','11103','11104','11105','11106','11109','11120'])

In [4]:
dat.columns

Index(['BORO', 'BLOCK', 'LOT', 'EASE', 'TC', 'FMV', 'UNITS', 'BC', 'SQFT',
       'ZIP', 'LAND_AREA', 'is_coop_or_condo', 'is_condo', 'is_coop', 'fy',
       'TCfull', 'lic_proper', 'lic_broad'],
      dtype='object')

In [5]:
landareas_byzip = dat.loc[dat['fy']==2024].groupby('ZIP',as_index=False)['LAND_AREA'].sum()

In [6]:
grossqft_byzipfy = dat.groupby(['fy','ZIP'],as_index=False)['SQFT'].sum()

In [7]:
grossqft_byzipfy.head()

,fy,ZIP,SQFT
0,2009,0.0,59380309.0
1,2009,10001,77915039.0
2,2009,10002,43042234.0
3,2009,10003,59025271.0
4,2009,10004,44273953.0


In [8]:
# pivot grossqft to get change
grossqft_byzipfy_pivot = grossqft_byzipfy.pivot(index = 'ZIP',columns='fy', values = 'SQFT')

In [9]:
grossqft_byzipfy_pivot.head()

fy,2009,2024
ZIP,,
,NaN,547171.0
0,NaN,51675412.0
0.0,59380309.0,NaN
00000,NaN,4671.0
10001,77915039.0,119503132.0


In [10]:
grossqft_byzipfy_pivot['delta'] = grossqft_byzipfy_pivot[2024] - grossqft_byzipfy_pivot[2009]

In [11]:
grossqft_byzipfy_pivot.head()

fy,2009,2024,delta
ZIP,,,
,NaN,547171.0,NaN
0,NaN,51675412.0,NaN
0.0,59380309.0,NaN,NaN
00000,NaN,4671.0,NaN
10001,77915039.0,119503132.0,41588093.0


In [12]:
grossqft_byzipfy_pivot = grossqft_byzipfy_pivot.dropna()

In [13]:
grossqft_byzipfy_pivot.head()

fy,2009,2024,delta
ZIP,,,
10001,77915039.0,119503132.0,41588093.0
10002,43042234.0,55900609.0,12858375.0
10003,59025271.0,63956240.0,4930969.0
10004,44273953.0,37097035.0,-7176918.0
10005,30116393.0,32581263.0,2464870.0


In [14]:
zip_summs = landareas_byzip.join(grossqft_byzipfy_pivot,on='ZIP')

In [15]:
zip_summs.head()

,ZIP,LAND_AREA,2009,2024,delta
0,,4.013860e+05,NaN,NaN,NaN
1,0,1.185231e+09,NaN,NaN,NaN
2,00000,2.053500e+04,NaN,NaN,NaN
3,10001,3.508351e+07,77915039.0,119503132.0,41588093.0
4,10002,3.346651e+08,43042234.0,55900609.0,12858375.0


In [16]:
zip_summs = zip_summs.dropna()

In [17]:
zip_summs.head()

,ZIP,LAND_AREA,2009,2024,delta
3,10001,35083506.0,77915039.0,119503132.0,41588093.0
4,10002,334665100.0,43042234.0,55900609.0,12858375.0
5,10003,82757664.0,59025271.0,63956240.0,4930969.0
6,10004,23274788.0,44273953.0,37097035.0,-7176918.0
7,10005,25488277.0,30116393.0,32581263.0,2464870.0


In [18]:
zip_summs['delta_per_landarea'] = zip_summs['delta']/zip_summs['LAND_AREA']

In [19]:
zip_summs['lic_proper'] = zip_summs['ZIP']=='11101'
zip_summs['lic_broad'] = zip_summs['ZIP'].isin(['11101','11102','11103','11104','11105','11106','11109','11120'])

In [20]:
zip_summs.head()

,ZIP,LAND_AREA,2009,2024,delta,delta_per_landarea,lic_proper,lic_broad
3,10001,35083506.0,77915039.0,119503132.0,41588093.0,1.185403,False,False
4,10002,334665100.0,43042234.0,55900609.0,12858375.0,0.038422,False,False
5,10003,82757664.0,59025271.0,63956240.0,4930969.0,0.059583,False,False
6,10004,23274788.0,44273953.0,37097035.0,-7176918.0,-0.308356,False,False
7,10005,25488277.0,30116393.0,32581263.0,2464870.0,0.096706,False,False


In [22]:
zip_summs.sort_values(by='delta_per_landarea',ascending=False).head(50)

,ZIP,LAND_AREA,2009,2024,delta,delta_per_landarea,lic_proper,lic_broad
85,10281,280175.0,2651866.0,5819649.0,3167783.0,11.306444,False,False
135,11109,906824.0,3290605.0,5394068.0,2103463.0,2.319593,False,True
3,10001,35083506.0,77915039.0,119503132.0,41588093.0,1.185403,False,False
9,10007,14106122.0,27662530.0,42058868.0,14396338.0,1.020574,False,False
36,10036,30100013.0,85542506.0,104235682.0,18693176.0,0.621035,False,False
99,10451,23919177.0,36653736.0,49859095.0,13205359.0,0.552082,False,False
42,10044,6776394.0,8358011.0,11853179.0,3495168.0,0.515786,False,False
103,10455,15600212.0,20643364.0,28131210.0,7487846.0,0.479984,False,False
151,11217,26788536.0,31780590.0,42981658.0,11201068.0,0.418129,False,False
18,10018,11432310.0,52771069.0,57545767.0,4774698.0,0.417649,False,False


In [23]:
zip_summs.sort_values(by='delta',ascending=False).head(50) # lic_proper is number 3 when ordered by grosqft delta, 2009->2024

,ZIP,LAND_AREA,2009,2024,delta,delta_per_landarea,lic_proper,lic_broad
3,10001,35083506.0,77915039.0,119503132.0,41588093.0,1.185403,False,False
136,11201,182108952.0,81973006.0,118660026.0,36687020.0,0.201456,False,False
129,11101,127501990.0,58697128.0,94662598.0,35965470.0,0.282078,True,True
16,10016,68848479.0,77475914.0,96306785.0,18830871.0,0.273512,False,False
36,10036,30100013.0,85542506.0,104235682.0,18693176.0,0.621035,False,False
23,10023,107170554.0,74307900.0,88910391.0,14602491.0,0.136255,False,False
140,11206,41001020.0,44008223.0,58445523.0,14437300.0,0.352121,False,False
9,10007,14106122.0,27662530.0,42058868.0,14396338.0,1.020574,False,False
180,11354,132762357.0,39072144.0,52824315.0,13752171.0,0.103585,False,False
99,10451,23919177.0,36653736.0,49859095.0,13205359.0,0.552082,False,False
